In [27]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import re
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.svm import SVC
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, roc_auc_score,confusion_matrix

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [1]:
from pan20_verif_evaluator import auc,c_at_1,f1,f_05_u_score

In [4]:
def normal(data):
    scaler = MinMaxScaler()
    return scaler.fit_transform(data) 

In [ ]:
# code from pan20_verif_evaluator.py
def auc(true_y, pred_y):
    
    try:
        return roc_auc_score(true_y, pred_y)
    except ValueError:
        return 0.0

def c_at_1(true_y, pred_y, threshold=0.5):
    

    n = float(len(pred_y))
    nc, nu = 0.0, 0.0

    for gt_score, pred_score in zip(true_y, pred_y):
        if pred_score == 0.5:
            nu += 1
        elif (pred_score > 0.5) == (gt_score > 0.5):
            nc += 1.0
    
    return (1 / n) * (nc + (nu * nc / n))


def f1(true_y, pred_y):
    
    true_y_filtered, pred_y_filtered = [], []

    for true, pred in zip(true_y, pred_y):
        if pred != 0.5:
            true_y_filtered.append(true)
            pred_y_filtered.append(pred)
    
    pred_y_filtered = binarize(pred_y_filtered)

    return f1_score(true_y_filtered, pred_y_filtered)


def f_05_u_score(true_y, pred_y, pos_label=1, threshold=0.5):
   

    pred_y = binarize(pred_y)

    n_tp = 0
    n_fn = 0
    n_fp = 0
    n_u = 0

    for i, pred in enumerate(pred_y):
        if pred == threshold:
            n_u += 1
        elif pred == pos_label and pred == true_y[i]:
            n_tp += 1
        elif pred == pos_label and pred != true_y[i]:
            n_fp += 1
        elif true_y[i] == pos_label and pred != true_y[i]:
            n_fn += 1

    return (1.25 * n_tp) / (1.25 * n_tp + 0.25 * (n_fn + n_u) + n_fp)

In [30]:

fpth = ['feature/feature_avg_sentence_length2max.txt','feature/feature_avg_word_length2max.txt','feature/feature_noun.txt',
       'feature/feature_pronoun.txt','feature/feature_adjective.txt','feature/feature_numeral.txt','feature/feature_verb.txt',
       'feature/feature_adverb.txt','feature/feature_end.txt','feature/feature_comma.txt','feature/feature_mispelling.txt','feature/feature_2gram.txt',
        'feature/feature_3gram.txt','feature/feature_4gram.txt','feature/feature_tfidf_cos.txt','feature/feature_yule.txt',
        'feature/feature_lexical_richness.txt','feature/feature_punc3gram.txt']
feature = np.zeros((52601,len(fpth)))
feature_nor = np.zeros((52601,len(fpth)))

for i in range(0,len(fpth)):
    feature_nor[:,i] = normal(np.loadtxt(fpth[i]).reshape(-1,1)).reshape(-1)


for i in range(0,len(fpth)):
    feature[:,i] = np.loadtxt(fpth[i])

In [31]:
target = pd.read_csv('data/info_truth.csv',encoding = 'utf-8')

In [32]:
y = np.array(target.same)

In [33]:
X_train,X_test, y_train, y_test = train_test_split(feature,y,test_size=0.3, random_state = 0, stratify = y)

In [34]:
(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

((36820, 18), (15781, 18), (36820,), (15781,))

# Random Forest

In [36]:
# 18

RF = RandomForestClassifier(n_estimators=400,random_state=11)
RF.fit(X_train,y_train)
y_pre = RF.predict(X_test)
#print("RF")
#print(classification_report(y_test,y_pre))
#print("AC",accuracy_score(y_test,y_pre))

print("F1",round(f1(y_test,y_pre),3))
print("auc",round(auc(y_test,y_pre),3))
print("c@1",round(c_at_1(y_test,y_pre),3))
print("F_0.5u",round(f_05_u_score(y_test,y_pre),3))

F1 0.814
auc 0.808
c@1 0.807
F_0.5u 0.825


In [10]:
# parameter change

RF = RandomForestClassifier(n_estimators=400,random_state=11)
RF.fit(X_train,y_train)
y_pre = RF.predict(X_test)
#print("RF")
#print(classification_report(y_test,y_pre))
#print("AC",accuracy_score(y_test,y_pre))

print("F1",round(f1(y_test,y_pre),3)
print("auc",round(auc(y_test,y_pre),3)
print("c@1",round(c_at_1(y_test,y_pre),3)
print("F_0.5u",round(f_05_u_score(y_test,y_pre),3)

RF
              precision    recall  f1-score   support

           0       0.78      0.82      0.80      7430
           1       0.83      0.79      0.81      8351

    accuracy                           0.81     15781
   macro avg       0.81      0.81      0.81     15781
weighted avg       0.81      0.81      0.81     15781

AC 0.8074266523034028
F1 0.8136840169210963
auc 0.8082194361681364
c@1 0.8074266523034028
F_0.5u 0.8255579607374786


In [63]:
#joblib.dump(RF, 'model/RF.pkl')

In [15]:
print("F1",round(f1(y_test,y_pre),3))
print("auc",round(auc(y_test,y_pre),3))
print("c@1",round(c_at_1(y_test,y_pre),3))
print("F_0.5u",round(f_05_u_score(y_test,y_pre),3))
#print((round(f1(y_test,y_pre),3)+round(auc(y_test,y_pre),3)+round(c_at_1(y_test,y_pre),3)+round(f_05_u_score(y_test,y_pre),3))/4)))

F1 0.814
auc 0.808
c@1 0.807
F_0.5u 0.826


In [115]:
# parameter change

RF = RandomForestClassifier(n_estimators=400,random_state=11)
RF.fit(X_train,y_train)
y_pre = RF.predict(X_test)
print("RF")
print(classification_report(y_test,y_pre))
print("AC",accuracy_score(y_test,y_pre))

print("F1 score",f1_score(y_test,y_pre))
print("Recall",recall_score(y_test,y_pre))
print("Precision",precision_score(y_test,y_pre))
print("Accuracy",accuracy_score(y_test,y_pre))

RF
              precision    recall  f1-score   support

           0       0.77      0.82      0.79      7430
           1       0.83      0.79      0.81      8351

    accuracy                           0.80     15781
   macro avg       0.80      0.80      0.80     15781
weighted avg       0.80      0.80      0.80     15781

AC 0.8004562448514035


In [123]:
print("F1 score",f1_score(y_test,y_pre))
print("Recall score",recall_score(y_test,y_pre))
print("Precision score",precision_score(y_test,y_pre))
print("Accuracy score",accuracy_score(y_test,y_pre))
print("Auc score",roc_auc_score(y_test,y_pre))

F1 score 0.8068216673823692
Recall score 0.7874506047179979
Precision score 0.8271698113207547
Accuracy score 0.8004562448514035
Auc score 0.8012623144720541


In [ ]:
RF.feature_importances_

In [120]:
confusion_matrix(y_test, y_pre)

array([[6056, 1374],
       [1775, 6576]])

In [112]:
# parameter change

RF = RandomForestClassifier(n_estimators=100,random_state=11)
RF.fit(X_train,y_train)
y_pre = RF.predict(X_test)
print("RF")
print(classification_report(y_test,y_pre))
print("AC",accuracy_score(y_test,y_pre))
print

RF
              precision    recall  f1-score   support

           0       0.77      0.82      0.79      7430
           1       0.83      0.79      0.81      8351

    accuracy                           0.80     15781
   macro avg       0.80      0.80      0.80     15781
weighted avg       0.80      0.80      0.80     15781

AC 0.8004562448514035


In [84]:
# norma; +tfidf +2 +yule +sentence lengthmax +word lengthmax + lexical richness

RF = RandomForestClassifier(n_estimators=10,random_state=11)
RF.fit(X_train,y_train)
y_pre = RF.predict(X_test)
print("RF")
print(classification_report(y_test,y_pre))
print("AC",accuracy_score(y_test,y_pre))

RF
              precision    recall  f1-score   support

           0       0.75      0.83      0.79      7430
           1       0.83      0.75      0.79      8351

    accuracy                           0.79     15781
   macro avg       0.79      0.79      0.79     15781
weighted avg       0.79      0.79      0.79     15781

AC 0.7865787972878778


In [66]:
# norma; +tfidf +2 +yule +sentence lengthmax +word lengthmax

RF = RandomForestClassifier(n_estimators=10,random_state=11)
RF.fit(X_train,y_train)
y_pre = RF.predict(X_test)
print("RF")
print(classification_report(y_test,y_pre))
print("AC",accuracy_score(y_test,y_pre))

RF
              precision    recall  f1-score   support

           0       0.74      0.82      0.78      7430
           1       0.82      0.75      0.78      8351

    accuracy                           0.78     15781
   macro avg       0.78      0.78      0.78     15781
weighted avg       0.79      0.78      0.78     15781

AC 0.7809391039858057


In [62]:
# no norma; +tfidf +2 +yule +sentence lengthmax +word lengthmax
# ！！！不做normalisation效果好
RF = RandomForestClassifier(n_estimators=10,random_state=11)
RF.fit(X_train,y_train)
y_pre = RF.predict(X_test)
print("RF")
print(classification_report(y_test,y_pre))
print("AC",accuracy_score(y_test,y_pre))

RF
              precision    recall  f1-score   support

           0       0.75      0.83      0.79      7430
           1       0.83      0.75      0.79      8351

    accuracy                           0.79     15781
   macro avg       0.79      0.79      0.79     15781
weighted avg       0.79      0.79      0.79     15781

AC 0.787909511437805


In [37]:
#+tfidf +2 +yule +sentence lengthmax
RF = RandomForestClassifier(n_estimators=10,random_state=11)
RF.fit(X_train,y_train)
y_pre = RF.predict(X_test)
print("RF")
print(classification_report(y_test,y_pre))
print("AC",accuracy_score(y_test,y_pre))

RF
              precision    recall  f1-score   support

           0       0.74      0.82      0.78      7430
           1       0.82      0.74      0.78      8351

    accuracy                           0.78     15781
   macro avg       0.78      0.78      0.78     15781
weighted avg       0.78      0.78      0.78     15781

AC 0.7802420632406057


In [29]:
#+tfidf +2 +yule
RF = RandomForestClassifier(n_estimators=10,random_state=11)
RF.fit(X_train,y_train)
y_pre = RF.predict(X_test)
print("RF")
print(classification_report(y_test,y_pre))
print("AC",accuracy_score(y_test,y_pre))

RF
              precision    recall  f1-score   support

           0       0.74      0.82      0.78      7430
           1       0.82      0.74      0.78      8351

    accuracy                           0.78     15781
   macro avg       0.78      0.78      0.78     15781
weighted avg       0.78      0.78      0.78     15781

AC 0.7801786959001331


In [18]:
#+tfidf +2
RF = RandomForestClassifier(n_estimators=10,random_state=11)
RF.fit(X_train,y_train)
y_pre = RF.predict(X_test)
print("RF")
print(classification_report(y_test,y_pre))
print("AC",accuracy_score(y_test,y_pre))

RF
              precision    recall  f1-score   support

           0       0.74      0.82      0.78      7430
           1       0.82      0.74      0.78      8351

    accuracy                           0.78     15781
   macro avg       0.78      0.78      0.78     15781
weighted avg       0.78      0.78      0.78     15781

AC 0.779735124516824


In [9]:
#+2gram 
RF = RandomForestClassifier(n_estimators=10,random_state=11)
RF.fit(X_train,y_train)
y_pre = RF.predict(X_test)
print("RF")
print(classification_report(y_test,y_pre))
print("AC",accuracy_score(y_test,y_pre))

RF
              precision    recall  f1-score   support

           0       0.73      0.81      0.77      7430
           1       0.81      0.73      0.77      8351

    accuracy                           0.77     15781
   macro avg       0.77      0.77      0.77     15781
weighted avg       0.77      0.77      0.77     15781

AC 0.767505227805589


In [80]:
#no 2
RF = RandomForestClassifier(n_estimators=10,random_state=11)
RF.fit(X_train,y_train)
y_pre = RF.predict(X_test)
print("RF")
print(classification_report(y_test,y_pre))
print("AC",accuracy_score(y_test,y_pre))

RF
              precision    recall  f1-score   support

           0       0.73      0.81      0.77      7430
           1       0.81      0.74      0.77      8351

    accuracy                           0.77     15781
   macro avg       0.77      0.77      0.77     15781
weighted avg       0.77      0.77      0.77     15781

AC 0.7704834928078068


# single feature random forest test

In [57]:
feama = np.zeros((18,5))
for i in range(18):
    X_train,X_test, y_train, y_test = train_test_split(feature[:,i].reshape(-1,1),y,test_size=0.3, random_state = 0, stratify = y)
    RF = RandomForestClassifier(n_estimators=40,random_state=11)
    RF.fit(X_train,y_train)
    y_pre = RF.predict(X_test)
    feama[i][0] = round(auc(y_test,y_pre),3)
    feama[i][1] = round(c_at_1(y_test,y_pre),3)
    feama[i][2] = round(f_05_u_score(y_test,y_pre),3)
    feama[i][3] = round(f1(y_test,y_pre),3)
    feama[i][4] = (feama[i][0] + feama[i][1] + feama[i][2] + feama[i][3]) /4
    

In [58]:
feama

array([[0.539  , 0.541  , 0.567  , 0.567  , 0.5535 ],
       [0.538  , 0.539  , 0.565  , 0.567  , 0.55225],
       [0.579  , 0.59   , 0.615  , 0.662  , 0.6115 ],
       [0.597  , 0.607  , 0.628  , 0.677  , 0.62725],
       [0.583  , 0.596  , 0.62   , 0.682  , 0.62025],
       [0.547  , 0.562  , 0.596  , 0.659  , 0.591  ],
       [0.613  , 0.624  , 0.641  , 0.692  , 0.6425 ],
       [0.592  , 0.603  , 0.625  , 0.679  , 0.62475],
       [0.624  , 0.633  , 0.648  , 0.69   , 0.64875],
       [0.665  , 0.672  , 0.682  , 0.717  , 0.684  ],
       [0.569  , 0.582  , 0.609  , 0.664  , 0.606  ],
       [0.572  , 0.574  , 0.597  , 0.598  , 0.58525],
       [0.635  , 0.636  , 0.657  , 0.654  , 0.6455 ],
       [0.676  , 0.674  , 0.698  , 0.678  , 0.6815 ],
       [0.568  , 0.57   , 0.594  , 0.597  , 0.58225],
       [0.542  , 0.544  , 0.568  , 0.566  , 0.555  ],
       [0.54   , 0.542  , 0.568  , 0.569  , 0.55475],
       [0.635  , 0.636  , 0.657  , 0.654  , 0.6455 ]])

# logstic regression

In [59]:
X_train,X_test, y_train, y_test = train_test_split(feature_nor,y,test_size=0.3, random_state = 0, stratify = y)

In [60]:

clf = LogisticRegression(penalty='l2')
clf.fit(X_train,y_train)
y_pre = clf.predict(X_test)

In [61]:
print("F1",round(f1(y_test,y_pre),3))
print("auc",round(auc(y_test,y_pre),3))
print("c@1",round(c_at_1(y_test,y_pre),3))
print("F_0.5u",round(f_05_u_score(y_test,y_pre),3))
#print((round(f1(y_test,y_pre),3)+round(auc(y_test,y_pre),3)+round(c_at_1(y_test,y_pre),3)+round(f_05_u_score(y_test,y_pre),3))/4)))

F1 0.77
auc 0.739
c@1 0.743
F_0.5u 0.746


In [62]:
joblib.dump(clf, 'model/LR.pkl')

['model/LR.pkl']

In [86]:
#不正则化结果超级低
clf = LogisticRegression(penalty='l2')
clf.fit(X_train,y_train)
y_pre = clf.predict(X_test)
print("LR")
print(classification_report(y_test,y_pre))
print("AC",accuracy_score(y_test,y_pre))

LR
              precision    recall  f1-score   support

           0       0.52      0.79      0.63      7430
           1       0.66      0.35      0.46      8351

    accuracy                           0.56     15781
   macro avg       0.59      0.57      0.54     15781
weighted avg       0.59      0.56      0.54     15781

AC 0.5590900449908117


# decision tree

In [64]:

clf = tree.DecisionTreeClassifier()
clf.fit(X_train,y_train)
y_pre = clf.predict(X_test)
print("F1",round(f1(y_test,y_pre),3))
print("auc",round(auc(y_test,y_pre),3))
print("c@1",round(c_at_1(y_test,y_pre),3))
print("F_0.5u",round(f_05_u_score(y_test,y_pre),3))

F1 0.73
auc 0.714
c@1 0.715
F_0.5u 0.731


In [65]:
joblib.dump(clf, 'model/DT.pkl')

['model/DT.pkl']

In [31]:

clf = tree.DecisionTreeClassifier()
clf.fit(X_train,y_train)
y_pre = clf.predict(X_test)
print("DT")
print(classification_report(y_test,y_pre))
print("AC",accuracy_score(y_test,y_pre))

DT
              precision    recall  f1-score   support

           0       0.69      0.69      0.69      7430
           1       0.73      0.73      0.73      8351

    accuracy                           0.71     15781
   macro avg       0.71      0.71      0.71     15781
weighted avg       0.71      0.71      0.71     15781

AC 0.7102845193587225


# SVM

In [66]:
# 6.4 335
clf = SVC(kernel='rbf', probability=True)
clf.fit(X_train,y_train)
y_pre = clf.predict(X_test)
print("F1",round(f1(y_test,y_pre),3))
print("auc",round(auc(y_test,y_pre),3))
print("c@1",round(c_at_1(y_test,y_pre),3))
print("F_0.5u",round(f_05_u_score(y_test,y_pre),3))

F1 0.784
auc 0.759
c@1 0.762
F_0.5u 0.766


In [26]:
joblib.dump(clf, 'model/svm.pkl')

767.75

In [72]:
clf = SVC(kernel='rbf', probability=True)
clf.fit(X_train,y_train)
y_pre = clf.predict(X_test)
print("SVM")
print(classification_report(y_test,y_pre))
print("AC",accuracy_score(y_test,y_pre))

SVM
              precision    recall  f1-score   support

           0       0.76      0.71      0.73      7430
           1       0.76      0.80      0.78      8351

    accuracy                           0.76     15781
   macro avg       0.76      0.76      0.76     15781
weighted avg       0.76      0.76      0.76     15781

AC 0.7573664533299538


In [ ]:
#joblib.dump(RF, 'model/LR.pkl')